In [23]:
import boto3
import pandas as pd

In [24]:
sm = boto3.Session().client('runtime.sagemaker')

# Test endpoint

In [31]:
def infer(input_data):
    response = sm.invoke_endpoint(EndpointName=endpoint, ContentType='text/csv', Accept='text/csv', Body=input_data)
    prediction = response['Body'].read().decode("utf-8")
    return prediction

In [32]:
endpoint = 'bank1'
data = '17.0,1.0,1.0,1.0,0.0,916.0,1.0,0.0,0.0,8.0,5.0,277.0,0.0,0.0,0.0,3.0'

prediction = infer(data)
print('prediction:', prediction)

prediction: 0.0



# Check accuracy with test data

In [33]:
DATA_PATH = 'data/'
df = pd.read_csv(DATA_PATH+'bank-test.csv')

In [34]:
tp = tn = fp = fn = 0

for index, row in df.iterrows():
    label = float(row['16'])
    input_data = ','.join(row.drop('16').astype(str).tolist())
    prediction = float(infer(input_data))
        
    if label == 1.0:
        if prediction == 1.0:
            tp=tp+1
        else:
            fn=fn+1
    else:
        if prediction == 0.0:
            tn=tn+1
        else:
            fp=fp+1

In [35]:
print ("%d %d" % (tn, fp))
print ("%d %d" % (fn, tp))

accuracy  = (tp+tn)/(tp+tn+fp+fn)
precision = tp/(tp+fp)
recall    = tn/(tn+fn)
f1        = (2*precision*recall)/(precision+recall)

print ("%.4f %.4f %.4f %.4f" % (accuracy, precision, recall, f1))

7678 304
551 510
0.9055 0.6265 0.9330 0.7497
